# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [56]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats


df=pd.read_csv( 'homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [58]:
df["timestamp"].min()

'2016-09-24 17:42:27.839496'

In [59]:
df["timestamp"].max()

'2017-01-18 10:24:08.629327'

In [60]:
df["id"].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [61]:
df[df["id"]==322257]

,timestamp,id,group,action
6969,2017-01-01 18:59:48.977251,322257,control,view
6970,2017-01-01 19:01:17.261656,322257,control,click


### Observation
probability that users who clicked also viewed since unique ids as 6328 yet 8188 entries

In [62]:
df["group"].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [63]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [64]:
#checking for duplicates in DataFrame
duplicates_df=df[df.duplicated(subset=["action","id"])]
duplicates_df


,timestamp,id,group,action


In [65]:
clicked=df[df["action"]=="click"]
viewed=df[df["action"]=="view"]

#Number who viewed but did not click
viewed_only=len(viewed)-len(clicked)
print(f"Viewed but did not click: ",viewed_only)


#confirming unique_ids
viewed_only + len(clicked)== len(df["id"].unique())

Viewed but did not click:  4468


True

In [66]:
obser_exp=df[(df["group"]=="control") & (df["group"]=="experiment")]
obser_exp

,timestamp,id,group,action


### Observations:
1. We can see that there are 8188 entries in the Dataframe
2. Timeframe is between 2016-09-24 and 2017-01-18
3. We however notice that there are 6328 unique IDs which indicates either duplicates or an ID performed two different events
4. No duplicates present hence indicating an ID performed different events i.e. clicked and viewed
5. Checked the number of ID users who viewed but did not click and confirmed against unique ID entries
6. There are no overlaps between the observed and experimental groups


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [67]:
#H0: Their was no dofference between the efficiency of the experiment group and the control group
#Ha: The experimental group homepage was more effective than the control group homepage

#   The experiment and control groups are independent groups where the sample sizes are different,
#   We shall hence use a Welch T-Test to carry out the hypothesis testing at an alpha of 0.05


In [69]:
df["click"]=(df["action"]=="click").apply(int)
df["view"]=(df["action"]=="view").apply(int)

In [70]:
df[:20]

,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,0,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,0,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,0,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,0,1


In [71]:
# df["click"]=0
# df["view"]=0

# df.head()

In [72]:
# df.loc[df["action"]=="click", "click"]=1
# df.loc[df["action"]!="click", "click"]=0

# df.loc[df["action"]=="view", "view"]=1
# df.loc[df["action"]!="view", "view"]=0
    

In [73]:
sum(df["click"]),sum(df["view"])

(1860, 6328)

In [74]:
df["group"].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [75]:
df.head()

,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1


In [76]:
control_df=df[df["group"]=="control"]
control_df=control_df[["id","view","click"]]
control_df=control_df.groupby("id").max()

experiment_df=df[df["group"]=="experiment"]
experiment_df=experiment_df[["id","view","click"]]
experiment_df=experiment_df.groupby("id").max()


control_df

,view,click
id,,
182994,1,1
183089,1,0
183248,1,1
183515,1,0
183524,1,0
...,...,...
936786,1,0
937003,1,0
937073,1,0


In [77]:
results=stats.ttest_ind(control_df["click"],experiment_df["click"], equal_var= False)
p_value=results.pvalue/2
p_value

0.004466402814337101

In [78]:
# the pvalue is smaller than the alpha value so ,
#we can say that it is statisticall significant to reject the hypothesis at a significance level of 0.05

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [79]:
#H0: Their was no dofference between the efficiency of the experiment group and the control group E==C
#Ha: The experimental group homepage was more effective than the control group homepage E!=C

In [80]:
#Your code here
control_rate=sum(control_df["click"])/len(control_df)
print(control_rate)


#expereiment same rate as control
experiment_number_alpha=control_rate*len(experiment_df)
experiment_number_alpha

0.2797118847539016


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [81]:

n = len(experiment_df)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [83]:
#Your code here
experiment_rate=experiment_df["click"].sum()
print(experiment_rate)

z_score=(experiment_rate-experiment_number_alpha)/ std
z_score

928


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [84]:
#Your code here
p_val= stats.norm.sf(z_score)
p_val

0.00012486528006951198

In [85]:
p_value

0.004466402814337101

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [86]:
"""
- It is evident that both produce p_values that are less than the alpha value and hence we can reject the hypothesis.
- However, the result is less than the orignal p_value calculated 


"""

'\n- It is evident that both produce p_values that are less than the alpha value and hence we can reject the hypothesis.\n- However, the result is less than the orignal p_value calculated \n\n\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.